In [1]:
import torch
torch.cuda.set_device(5)
print(torch.cuda.current_device())

5


In [2]:
%run ../../prepare_data.py
%run ../../architectures.py

In [3]:
batch_size = 32

In [4]:
train_loader, valid_loader, valid_dataset = get_chexpert_dataloaders(batch_size)

In [5]:
x, y = next(iter(train_loader))

In [6]:
x.shape, y.shape

(torch.Size([32, 3, 256, 256]), torch.Size([32, 5]))

In [7]:
model = resnet18(num_classes=5).cuda()

In [8]:
sum(p.numel() for p in model.parameters())

11179077

In [9]:
# %%time
# lrs, losses = LR_range_finder(model, train_loader, 
#                               loss_fn=F.binary_cross_entropy_with_logits, 
#                               binary=False, lr_high=0.05)
# plot_lr(lrs, losses)

# Training

In [10]:
widths = [1.0, 0.75, 0.5, 0.25]
depths = [[[[64, 2], [128, 2]], [[256, 2], [512, 1]]],
          [[[64, 2], [128, 2]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 1], [512, 1]]],
         ]

In [ ]:
data = []

for w in widths:
    for d in depths:
        d_s = sum(j[1] for i in d for j in i)
        print('width multiplier - %.3f depth multiplier - %.3f' % (w, d_s))
        model = resnet18(num_classes=5, width_mult=w, 
                         inverted_residual_setting1=d[0], 
                         inverted_residual_setting2=d[1]).cuda()
        
        p = sum(p.numel() for p in model.parameters())
        optimizer = create_optimizer(model, 0.001)
        score, t = train_triangular_policy(model, optimizer, train_loader, valid_loader, valid_dataset,
                                       loss_fn=F.binary_cross_entropy_with_logits, 
                                       dataset='chexpert', binary=False, max_lr=0.001, epochs=15)
        
        p = "/home/rimmanni/Medical_Images/Scaling_experiments/Chexpert/Resnet_" + str(w) + '_' + str(depths.index(d))
        save_model(model, p)
        data.append([w, d_s, score, p, t])
        print('')

width multiplier - 1.000 depth multiplier - 7.000
train_loss 0.411 val_loss 0.417 val_auc_score 0.633
----End of step 0:01:29.231922
train_loss 0.397 val_loss 0.417 val_auc_score 0.658
----End of step 0:01:29.793563
train_loss 0.391 val_loss 0.404 val_auc_score 0.674
----End of step 0:01:33.081681
train_loss 0.385 val_loss 0.403 val_auc_score 0.687
----End of step 0:01:34.369049
train_loss 0.381 val_loss 0.398 val_auc_score 0.686
----End of step 0:01:35.963838
train_loss 0.379 val_loss 0.418 val_auc_score 0.693
----End of step 0:01:36.583420
train_loss 0.375 val_loss 0.388 val_auc_score 0.706
----End of step 0:01:35.063999
train_loss 0.372 val_loss 0.401 val_auc_score 0.702
----End of step 0:01:34.811404
train_loss 0.369 val_loss 0.390 val_auc_score 0.702
----End of step 0:01:35.061466
train_loss 0.365 val_loss 0.382 val_auc_score 0.716
----End of step 0:01:35.066333
train_loss 0.362 val_loss 0.387 val_auc_score 0.716
----End of step 0:01:34.873348
train_loss 0.359 val_loss 0.379 val_a

train_loss 0.377 val_loss 0.403 val_auc_score 0.697
----End of step 0:01:20.665688
train_loss 0.374 val_loss 0.400 val_auc_score 0.703
----End of step 0:01:20.601409
train_loss 0.371 val_loss 0.385 val_auc_score 0.704
----End of step 0:01:20.682445
train_loss 0.368 val_loss 0.383 val_auc_score 0.707
----End of step 0:01:20.489084
train_loss 0.365 val_loss 0.385 val_auc_score 0.708
----End of step 0:01:20.591338
train_loss 0.362 val_loss 0.386 val_auc_score 0.706
----End of step 0:01:20.438862
train_loss 0.360 val_loss 0.379 val_auc_score 0.718
----End of step 0:01:20.316658
train_loss 0.357 val_loss 0.377 val_auc_score 0.719
----End of step 0:01:19.943016
train_loss 0.354 val_loss 0.378 val_auc_score 0.718
----End of step 0:01:20.162583
train_loss 0.354 val_loss 0.378 val_auc_score 0.719
----End of step 0:01:20.496334

width multiplier - 0.750 depth multiplier - 6.000
train_loss 0.413 val_loss 0.430 val_auc_score 0.620
----End of step 0:01:17.848270
train_loss 0.400 val_loss 0.414 val_

In [ ]:
columns = ['width_x', 'depth_x', 'val_score', 'params', 'time_per_epoch']
df = pd.DataFrame(data=data, columns=columns)

In [ ]:
df.to_csv("chexpert_resnet_13.csv", index=False)

In [ ]:
df_re = pd.read_csv('chexpert_resnet_13.csv')

In [ ]:
df_re.head()